In [1]:
import github3
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv()

github = github3.login(token=os.getenv('GITHUB_TOKEN'))

In [2]:
js_issues = github.issues_on(username='open-telemetry', repository='opentelemetry-js', state='open')

In [3]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor
import concurrent.futures


class Issue(BaseModel):
    title: str = Field(..., description="The title of the issue.")
    url: str = Field(..., description="The URL of the issue.")
    libraries: List[str] = Field(..., description="A list of libraries mentioned in the issue.")
    body: str = Field(..., description="The body of the issue.")
    positive_reactions: int = Field(..., description="The number of positive reactions on the issue.")
    negative_reactions: int = Field(..., description="The number of negative reactions on the issue.")
    inferred_sentiment: str = Field(..., description="The sentiment of the issue body (one of 'positive', 'negative', or 'neutral').")

client = instructor.from_openai(OpenAI())

responses = []

def process_issue(issue):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a system that parses GitHub issues and extracts data from them."
            },
            {
                "role": "user",
                "content": issue
            }
        ],
        response_model=Issue
    )
    return resp

json_issues = [issue.json() for issue in js_issues]

# Function to print the status
def print_status(completed, total):
    print(f"Processed {completed} of {total} items")

# Use concurrent.futures to process issues in parallel using threads
responses = []
total_tasks = len(json_issues)

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_issue, json_content): json_content for json_content in json_issues}
    completed_tasks = 0

    for future in concurrent.futures.as_completed(futures):
        try:
            response = future.result()
            responses.append(response)
        except Exception as exc:
            print(f'Generated an exception: {exc}')
        completed_tasks += 1
        print_status(completed_tasks, total_tasks)

Processed 0 of 298 items


In [ ]:
%pip install libhoney

In [ ]:
import libhoney
libhoney.init(writekey=os.getenv('HONEYCOMB_API_KEY'), dataset='otel-github-issues')
for response in responses:
    evt = libhoney.Event()
    evt.add_field('title', response.title)
    evt.add_field('url', response.url)
    evt.add_field('positive_reactions', response.positive_reactions)
    evt.add_field('negative_reactions', response.negative_reactions)
    evt.add_field('inferred_sentiment', response.inferred_sentiment)
    evt.add_field('body', response.body)
    for library in response.libraries:
        evt.add_field('library', library)
    evt.send()